In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.model_selection import cross_val_score

from sklearn.metrics import fbeta_score, make_scorer

In [0]:
base = "drive/My Drive/외부/공모전/[기상] AI프렌즈 시즌1 온도 추정 경진대회/"

data = pd.read_csv(os.path.join(base, "train.csv"))

### 똑같은 값만 나오는 센서 삭제

In [100]:
broken_cols = data.columns[data.max() == data.min()]
print(broken_cols)

for col in broken_cols:
  data = data.drop(columns=col)

Index(['X14', 'X16', 'X19'], dtype='object')


In [0]:
data['hour'] = (data.index % 144 / 6).astype('int')
data['minute'] = (data.index % 144 % 6).astype('int')

In [0]:
X_cols = ['hour', 'minute'] + [col for col in data.columns if 'X' in col]
Y_cols = ["Y{0:02}".format(i) for i in range(18)]

train = data[:4320]
val = data[4320:]

In [103]:
train_temp = pd.DataFrame(columns=X_cols + ['Y'])

for y in Y_cols:
  temp_data = train[X_cols + [y]]
  temp_data.rename(columns = {y : 'Y'}, inplace = True)
  train_temp = pd.concat([train_temp, temp_data], axis=0)

train = train_temp

# print(train.isnull().sum().sum())
# display(train)

0


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
for Y in Y_cols:
  del val[Y]
del val['id']

In [105]:
val.rename(columns = {'Y18' : 'Y'}, inplace = True)
all = pd.concat([train, val], axis=0)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Feature Selection

In [0]:
def forward_selection(clf, dataset, X_cols, Y_col='Y18'):
  sfs = SFS(clf, 
            k_features=len(X_cols), 
            forward=True, 
            floating=False, 
            scoring='neg_mean_squared_error',
            cv=10)

  sfs = sfs.fit(dataset[X_cols], dataset[Y_col])
  fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')
  plt.title('Sequential Forward Selection (w. StdErr)')
  plt.grid()
  plt.show()

  return sfs

def print_good_features(sfs):
  metric = sfs.get_metric_dict()
  avg_scores = dict()
  max_key = None

  for key in metric.keys():
    avg_scores[metric[key]['feature_names']] = metric[key]['avg_score']

  for key, score in sorted(avg_scores.items(), key=lambda x: x[1], reverse=True)[:2]:
    if max_key is None: max_key = key
    
    print("feature_names: {}, avg_score: {}".format(key, score))

    return max_key

def check_cross_val_score(clf, dataset, X_cols, Y_col='Y18', cv=5):
  scores = cross_val_score(clf, val[X_cols], val[Y_col], cv=cv)

  print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

def clf_train(clf, X, Y):
  clf.fit(X, Y)
  
  return clf

def mse_AIFrenz(y_true, y_pred):
    diff = abs(y_true - y_pred)
    
    less_then_one = np.where(diff < 1, 0, diff)
    
    score = np.average(np.average(less_then_one ** 2, axis = 0))
    
    return score

mse_score = make_scorer(mse_AIFrenz, greater_is_better=False)

### Ridge을 이용한 forward Seletion

In [0]:
sfs = forward_selection(Ridge(alpha=1.0), train, X_cols, 'Y')
features = print_good_features(sfs)

In [0]:
ridge_cols = list(features)

clf = clf_train(Ridge(alpha=1.0), train[ridge_cols], train['Y'])
print(f"val acc : {mse_AIFrenz(clf)}")
print(check_cross_val_score(Ridge(alpha=1.0), val, ridge_feature_name, 'Y'))

### Univariate Selection
- Ref: https://wikidocs.net/16599

In [36]:
from sklearn.feature_selection import SelectKBest, f_classif

selectK = SelectKBest(score_func=f_classif, k=8)
X = selectK.fit_transform(val[X_cols], val['Y'])

selectK_cols = list()

for col in X_cols:
  if list(val[col])[0] in X[0]:
    print("'{}'".format(col), end=", ")
    selectK_cols.append(col)
print()

clf = clf_train(Ridge(alpha=1.0), train[selectK_cols], train['Y'])
print(f"val acc : {mse_AIFrenz(clf)}")

'X00', 'X07', 'X12', 'X20', 'X30', 'X31', 'X32', 'X37', 


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 6 35] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


### Feature Importance

In [37]:
from sklearn.feature_selection import RFE

model = LinearRegression()
rfe = RFE(model, 8)
fit = rfe.fit(val[X_cols], val['Y'])

rfe_cols = np.array(X_cols)[fit.support_]

print(f"Num Features: {fit.n_features_}")
print(f"Selected Features: {np.array(X_cols)[fit.support_]}")
print(f"Feature Ranking: {np.array(X_cols)[fit.ranking_]}")

clf = clf_train(Ridge(alpha=1.0), train[rfe_cols], train['Y'])
print(f"val acc : {mse_AIFrenz(clf)}")

Num Features: 8
Selected Features: ['X01' 'X06' 'X08' 'X09' 'X27' 'X29' 'X33' 'X39']
Feature Ranking: ['X03' 'X24' 'X18' 'minute' 'X09' 'X28' 'X32' 'X01' 'minute' 'X22'
 'minute' 'minute' 'X12' 'X05' 'X25' 'X29' 'X30' 'X31' 'X13' 'X20' 'X17'
 'X00' 'X02' 'X21' 'X27' 'X08' 'minute' 'X11' 'minute' 'X15' 'X07' 'X23'
 'minute' 'X04' 'X26' 'X33' 'X06' 'X10' 'minute']


In [0]:
feature_cols = {'ridge': ridge_cols,
                 'rfe': rfe_cols,
                 'selectK' : selectK_cols}

### Model Selection

In [74]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid = { 'alpha': [0.01, 0.1, 1.0, 10.0, 100.0], 
              'fit_intercept': [True, False],
              'normalize': [True, False],
              'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
              'tol' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
              }

grid = GridSearchCV(Ridge(), param_grid, scoring=mse_score, cv=5)  
grid.fit(val[X_cols], val['Y'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
                         'fit_intercept': [True, False],
                         'normalize': [True, False],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg', 'sag', 'saga'],
                         'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(mse_AIFrenz, greater_is_better=False),
             verbose=0)

In [75]:
print(grid.best_params_)
print(grid.best_score_)

{'alpha': 100.0, 'fit_intercept': True, 'normalize': False, 'solver': 'sparse_cg', 'tol': 0.1}
-9.156287123536316


In [68]:
dir(grid)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',

### Ridge 예측하기

In [54]:
for name, X_cols in feature_cols.items():
  clf = Ridge(alpha=1.0, normalize=True)
  clf.fit(train_all[X_cols], train_all['Y'])

  print(f"{name} : {mse_AIFrenz(val['Y'], clf.predict(val[X_cols]))}")

clf = Ridge(alpha=1.0, normalize=True)
clf.fit(train_all[X_cols], train_all['Y'])

print(f"all : {mse_AIFrenz(val['Y'], clf.predict(val[X_cols]))}")

ridge : 30.099152505341728
rfe : 46.299891911694495
selectK : 12.815951466239385
all : 12.815951466239385


### lightbgm으로 예측하기

In [0]:
def train_lgb(dataset, X_cols, Y_col='Y'):
    lgb_train = lgb.Dataset(dataset[X_cols], label=dataset[Y_col])

    lgb_param = {
        'objective': 'regression',
        'metrics': 'mae',
        'learning_rate' : 0.01
    }

    cv_result = lgb.cv(
        lgb_param,
        lgb_train,
        num_boost_round=99999,
        nfold = 5,
        early_stopping_rounds=50,
        stratified=False,
        verbose_eval=10
    )

    lgb_model = lgb.train(
      lgb_param,
      lgb_train,
      num_boost_round=len(cv_result['l1-mean'])
    )

    return lgb_model

def get_val_accuracy(clf, X_cols, Y_col='Y'):
  pred = clf.predict(val[X_cols])
  mae = mse_AIFrenz(val[Y_col], pred)

  return mae

def submit(clf, X_cols, name="None"):
  submit = pd.read_csv(base + 'sample_submission.csv')
  test = pd.read_csv(base + "test.csv")
  submit['Y18'] = clf.predict(test[X_cols])

  submit.to_csv(os.path.join(base, "제출", name + ".csv"), index=None)

In [0]:
results = dict()

for name, features in feature_names.items():
  lgb_model = train_lgb(train, features, 'Y')

  val_accuracy = get_val_accuracy(lgb_model, features, 'Y')

  print(f"feature_name: {name}, val_accracy: {val_accuracy}")
  results[name] = val_accuracy

[10]	cv_agg's l1: 6.0694 + 0.0353323
[20]	cv_agg's l1: 5.65881 + 0.0330114
[30]	cv_agg's l1: 5.30298 + 0.0309959
[40]	cv_agg's l1: 4.99777 + 0.0285486
[50]	cv_agg's l1: 4.73671 + 0.0250791
[60]	cv_agg's l1: 4.51238 + 0.0218909
[70]	cv_agg's l1: 4.32188 + 0.0176035
[80]	cv_agg's l1: 4.16087 + 0.0143713
[90]	cv_agg's l1: 4.02363 + 0.0119378
[100]	cv_agg's l1: 3.90765 + 0.0105451
[110]	cv_agg's l1: 3.80745 + 0.00874396
[120]	cv_agg's l1: 3.72235 + 0.00722955
[130]	cv_agg's l1: 3.64955 + 0.00553418
[140]	cv_agg's l1: 3.58544 + 0.00464049
[150]	cv_agg's l1: 3.52947 + 0.00424269
[160]	cv_agg's l1: 3.48088 + 0.00417837
[170]	cv_agg's l1: 3.43764 + 0.00439187
[180]	cv_agg's l1: 3.40009 + 0.0040932
[190]	cv_agg's l1: 3.36761 + 0.00417764
[200]	cv_agg's l1: 3.33807 + 0.00450489
[210]	cv_agg's l1: 3.31173 + 0.00473816
[220]	cv_agg's l1: 3.28705 + 0.00453795
[230]	cv_agg's l1: 3.26497 + 0.00461915
[240]	cv_agg's l1: 3.24493 + 0.00429661
[250]	cv_agg's l1: 3.22665 + 0.00418475
[260]	cv_agg's l1: 3.

In [0]:
max_name = None

for name, accuracy in sorted(results.items(), key=lambda x: x[1], reverse=True):
  if max_name is None : max_name = name
  print(f"feature_name: {name}, val_accracy: {accuracy}")

feature_name: rfe, val_accracy: 3.7257219119329017
feature_name: lr, val_accracy: 3.1455835165717967
feature_name: ridge, val_accracy: 3.0667992522627148
feature_name: selectK, val_accracy: 2.0908644852840705


In [0]:
feature_names['selectK']

['X00', 'X07', 'X12', 'X20', 'X30', 'X31', 'X32', 'X37']

### Submit

In [0]:
total_data = pd.concat([train, val], axis=0)

In [0]:
lgb_model = train_lgb(total_data, feature_names[max_name], 'Y')
submit(lgb_model, feature_names[max_name], "lightbgm_allY=Y")

[10]	cv_agg's l1: 6.2212 + 0.018155
[20]	cv_agg's l1: 5.94456 + 0.0181608
[30]	cv_agg's l1: 5.69823 + 0.0184474
[40]	cv_agg's l1: 5.47938 + 0.0188786
[50]	cv_agg's l1: 5.2853 + 0.0191246
[60]	cv_agg's l1: 5.1146 + 0.0191522
[70]	cv_agg's l1: 4.96279 + 0.0194202
[80]	cv_agg's l1: 4.82798 + 0.0208015
[90]	cv_agg's l1: 4.70768 + 0.0204617
[100]	cv_agg's l1: 4.60004 + 0.0204065
[110]	cv_agg's l1: 4.50326 + 0.0206388
[120]	cv_agg's l1: 4.41684 + 0.0202653
[130]	cv_agg's l1: 4.33718 + 0.0206465
[140]	cv_agg's l1: 4.26428 + 0.0196357
[150]	cv_agg's l1: 4.19828 + 0.0198702
[160]	cv_agg's l1: 4.13849 + 0.0197528
[170]	cv_agg's l1: 4.08587 + 0.0201984
[180]	cv_agg's l1: 4.03778 + 0.0197599
[190]	cv_agg's l1: 3.99353 + 0.0193612
[200]	cv_agg's l1: 3.95271 + 0.0190216
[210]	cv_agg's l1: 3.91455 + 0.019057
[220]	cv_agg's l1: 3.87991 + 0.0189103
[230]	cv_agg's l1: 3.8475 + 0.0182186
[240]	cv_agg's l1: 3.81746 + 0.0186154
[250]	cv_agg's l1: 3.78919 + 0.0184152
[260]	cv_agg's l1: 3.76236 + 0.018896
[2

In [0]:
ridge = Ridge(alpha=1.0)
ridge.fit(total_data[feature_names['ridge']], total_data['Y'])

submit(ridge, feature_names['ridge'], "Ridge_allY=Y")

In [0]:
display(total_data)

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y,id,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.00,82.4,71.3,101.1,56.1,0.3,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.00,256.4,0.0,77.2,62.6,0.0,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.00,81.2,121.8,165.3,0.0,0.4,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.00,215.4,0.0,77.3,63.5,0.0,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.00,86.1,232.0,144.0,0.0,0.6,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.00,235.2,0.0,77.3,63.9,0.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.00,87.7,0.0,115.9,294.1,0.1,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.00,214.0,0.0,77.5,64.5,0.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.00,88.9,0.0,124.7,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.00,174.9,0.0,78.0,65.0,0.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,19.9,987.6,0.9,0.8,0.0,1006.9,987.7,21.7,1007.5,1007.4,2.5,23.88,95.3,333.1,274.9,275.5,0.9,89.7,10.5,1000.4,1007.5,0.0,0.0,1.2,1006.0,23.8,999.4,89.9,17.7,19.1,1007.5,22.16,218.6,0.0,82.3,58.6,0.5,21.5,4747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4748,19.9,987.6,0.5,0.7,0.0,1006.8,987.7,21.6,1007.5,1007.4,2.5,23.88,95.6,216.4,277.3,247.0,1.0,90.1,10.5,1000.3,1007.5,0.0,0.0,1.3,1006.0,23.6,999.4,91.3,17.7,19.2,1007.5,22.16,161.7,0.0,82.5,59.1,0.5,21.5,4748.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,1007.5,2.5,23.88,95.9,127.8,256.1,299.1,0.6,90.8,10.5,1000.4,1007.6,0.0,0.0,0.2,1006.0,23.6,999.5,90.2,17.8,19.2,1007.5,22.16,254.2,0.0,83.0,58.9,0.5,21.5,4749.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,1007.5,2.5,23.88,95.9,323.8,255.1,0.0,0.6,91.5,10.5,1000.4,1007.6,0.0,0.0,0.1,1006.0,23.3,999.5,90.1,17.7,19.3,1007.6,22.16,300.0,0.0,83.2,59.8,0.5,21.5,4750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
def train_lgb(dataset, X_cols, Y_col='Y18'):
    lgb_train = lgb.Dataset(dataset[X_cols], label=dataset[Y_col])

    lgb_param = {
        'objective': 'regression',
        'metrics': 'mae',
        'learning_rate' : 0.01
    }

    cv_result = lgb.cv(
        lgb_param,
        lgb_train,
        num_boost_round=99999,
        nfold = 5,
        early_stopping_rounds=50,
        stratified=False,
        verbose_eval=10
    )

    lgb_model = lgb.train(
      lgb_param,
      lgb_train,
      num_boost_round=len(cv_result['l1-mean'])
    )

    return lgb_model

In [0]:
lgb_train_data = lgb.Dataset(total_data[X_cols], label=total_data['Y'])

lgb_param = {
    'objective': 'regression',
    'metrics': 'mae',
    'learning_rate' : 0.01
}

cv_result = lgb.cv(
    lgb_param,
    lgb_train_data,
    num_boost_round=99999,
    nfold = 5,
    early_stopping_rounds=50,
    stratified=False,
    verbose_eval=10
)

[10]	cv_agg's l1: 6.01514 + 0.0157208
[20]	cv_agg's l1: 5.55824 + 0.014181
[30]	cv_agg's l1: 5.15986 + 0.0131625
[40]	cv_agg's l1: 4.81611 + 0.0126397
[50]	cv_agg's l1: 4.52197 + 0.0123673
[60]	cv_agg's l1: 4.2713 + 0.011604
[70]	cv_agg's l1: 4.05866 + 0.0108586
[80]	cv_agg's l1: 3.87712 + 0.0106206
[90]	cv_agg's l1: 3.72129 + 0.0108849
[100]	cv_agg's l1: 3.58694 + 0.0115497
[110]	cv_agg's l1: 3.47043 + 0.0123174
[120]	cv_agg's l1: 3.3695 + 0.0132596
[130]	cv_agg's l1: 3.28208 + 0.0141653
[140]	cv_agg's l1: 3.2062 + 0.0150891
[150]	cv_agg's l1: 3.13994 + 0.0158303
[160]	cv_agg's l1: 3.08236 + 0.0165291
[170]	cv_agg's l1: 3.03216 + 0.0170367
[180]	cv_agg's l1: 2.98857 + 0.0174042
[190]	cv_agg's l1: 2.95051 + 0.0177787
[200]	cv_agg's l1: 2.91724 + 0.0181679
[210]	cv_agg's l1: 2.88819 + 0.0184604
[220]	cv_agg's l1: 2.86292 + 0.0187533
[230]	cv_agg's l1: 2.84077 + 0.0190563
[240]	cv_agg's l1: 2.82135 + 0.0193187
[250]	cv_agg's l1: 2.80434 + 0.0196195
[260]	cv_agg's l1: 2.78943 + 0.0197876


In [0]:
lgb_model = lgb.train(
  lgb_param,
  lgb_train,
  num_boost_round=len(cv_result['l1-mean'])
)

In [0]:
lgb_model = train_lgb(total_data, feature_names[max_name], 'Y')
submit(lgb_model, feature_names[max_name], "lightbgm_allY=Y")

lgb.LGBMRegressor

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_doc',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 'best_iteration_',
 'best_score_',
 'booster_',
 'evals_result_',
 'feature_importances_',
 'fit',
 'get_params',
 'n_features_',
 'objective_',
 'predict',
 'score',
 'set_params']